# 네이버 영화 랭킹 크롤링

- toc: true
- branch: master
- badges: true
- comments: true
- author: Kim Jeewoo
- categories: [Crawling, R]

# 네이버 영화 랭킹 크롤링

- 네이버 영화 순위를 크롤링한다.

# Code

In [2]:
# 라이브러리를 불러옴, 존재하지 않으면 설치합니다.
if (!require(rvest)) install.packages('rvest')
library(rvest)

if (!require(tidyverse)) install.packages('tidyverse')
library(tidyverse)


# 날짜와 페이지를 입력하면 조건에 따른 영화 코드를 반환하는 함수를 정의함
get_movie_code <- function(date, page){ # ex) date = 20220502, page = 1
    
    base_url <- 'https://movie.naver.com/movie/sdb/rank/rmovie.naver?sel=pnt&date='
    target_url <- paste0(base_url, date, '&page=', page)
    
    tables <- target_url %>%
        read_html(encoding = 'UTF-8') %>%
        html_nodes('table')
    
    hrefs <- tables[[1]] %>%
        html_nodes('a') %>%
        html_attr('href')
    
    hrefs_odd <- hrefs[c(TRUE, FALSE)] # 같은 코드이나 서로 다른링크가 짝으로 존재함을 확인했다. 홀수번째 원소만 인덱싱한다.
    codes <- substr(hrefs_odd, unlist(gregexpr('=', hrefs_odd)) + 1, nchar(hrefs_odd)) # 다섯자리 코드와 여섯자리 코드가 혼재하므로 등호를 기준으로 인덱싱한다.
    
    return(codes)
}

# 영화의 코드를 입력하면 영화의 정보를 반환하는 함수를 정의함
get_movie_info <- function(code){
    base_url <- 'https://movie.naver.com/movie/bi/mi/point.nhn?code='
    target_url <- paste0(base_url, code)
    html <- read_html(target_url)
    
    title_unclean <- html %>%
    html_nodes("title") %>%
    html_text()

    title <- substr(title_unclean, 1, unlist(gregexpr(' : 네이버 영화', title_unclean))-1)
    exist <- html %>% html_nodes('dl[class=info_spec]') %>% html_nodes('dt') %>% html_text()

    steps <- html %>%
        html_nodes('dl[class=info_spec]') %>%
        html_nodes('dd')

    # 개요, 감독, 출연, 등급 중 결측값이 존재하는 경우를 대비함
    step1 = NA
    step2 = NA
    step3 = NA
    step4 = NA
    for (i in 1:length(exist)){
        if (exist[i] == '개요()'){
            step1_unclean <- steps[i] %>%
                html_nodes('p') %>%
                    html_nodes('span') %>%
                html_text()
            step1 <- gsub('\\t|\\n|\\r', '', step1_unclean)
        }else if (exist[i] == '감독'){
            step2 <- steps[i] %>%
                html_nodes('p') %>%
                html_text()
        }else if (exist[i] == '출연'){
            step3 <- steps[i] %>%
                html_text()
        }else if (exist[i] == '등급'){
            step4_unclean <- steps[i] %>%
                html_nodes('p') %>%
                html_text()
            step4 <- gsub('\\t|\\n|\\r', '', step4_unclean)
        }
    }
    
    if (length(step1) == 3){ # 개봉일자가 존재하지 않는 경우 결측값으로 처리함 ex)먼 훗날 우리
        step1 = c(step1, NA)
    }
    
    tdt <- html %>%
        html_nodes('div[class=viewing_graph]')
    
    # 성별, 나이별 관람추이가 존재하지 않는 경우 관람객 통계가 존재하지 않으므로 결측값으로 처리함
    if (length(tdt) == 0){
        audience_age_10 <- NA
        audience_age_20 <- NA
        audience_age_30 <- NA
        audience_age_40 <- NA
        audience_age_50 <- NA
        audience_score <- NA
        audience_count <- NA
        audience_male <- NA
        audience_female <- NA
        audience_10 <- NA
        audience_20 <- NA
        audience_30 <- NA
        audience_40 <- NA
        audience_50 <- NA
    } else {
        audi_age <- html %>%
            html_nodes('strong[class=graph_percent]') %>%
            html_text()
        audience_age_10 <- audi_age[1]
        audience_age_20 <- audi_age[2]
        audience_age_30 <- audi_age[3]
        audience_age_40 <- audi_age[4]
        audience_age_50 <- audi_age[5]

        audience_score <- html %>% 
            html_nodes('div[class=grade_audience]') %>%
            html_nodes('div[class=star_score]') %>%
            html_nodes('em') %>%
            html_text() %>% paste(collapse='')

        audience_count <- html %>% 
            html_nodes('div[class=grade_audience]') %>%
            html_nodes('span[class=user_count]') %>%
            html_nodes('em') %>%
            html_text() %>% paste(collapse='')
        
        audience_male <- (html %>%
            html_nodes('div[class=graph_area]') %>%
            html_nodes('div[class=grp_male]') %>%
            html_nodes('strong[class=graph_point]') %>%
            html_text())[2]

        audience_female <- (html %>%
            html_nodes('div[class=graph_area]') %>%
            html_nodes('div[class=grp_female]') %>%
            html_nodes('strong[class=graph_point]') %>%
            html_text())[2]

        audience_age <- html %>%
            html_nodes('div[class=grp_age]') %>%
            html_nodes('strong[class=graph_point]') %>%
            html_text()
        
        audience_10 <- audience_age[6]
        audience_20 <- audience_age[7]
        audience_30 <- audience_age[8]
        audience_40 <- audience_age[9]
        audience_50 <- audience_age[10]
        
    }
    
    netizen_score <- html %>% 
        html_nodes('div[class=grade_netizen]') %>%
        html_nodes('div[class=star_score]') %>%
        html_nodes('em') %>%
        html_text() %>% paste(collapse='')
    
    netizen_count <- html %>% 
        html_nodes('div[class=grade_netizen]') %>%
        html_nodes('span[class=user_count]') %>%
        html_nodes('em') %>%
        html_text() %>% paste(collapse='')
    
    ntz_male <- (html %>%
        html_nodes('div[class=graph_area]') %>%
        html_nodes('div[class=grp_male]') %>%
        html_nodes('strong[class=graph_point]') %>%
        html_text())[1]
    
    ntz_female <- (html %>%
        html_nodes('div[class=graph_area]') %>%
        html_nodes('div[class=grp_female]') %>%
        html_nodes('strong[class=graph_point]') %>%
        html_text())[1]
    
    ntz_age <- html %>%
        html_nodes('div[class=grp_age]') %>%
        html_nodes('strong[class=graph_point]') %>%
        html_text()
    ntz_10 <- ntz_age[1]
    ntz_20 <- ntz_age[2]
    ntz_30 <- ntz_age[3]
    ntz_40 <- ntz_age[4]
    ntz_50 <- ntz_age[5]
    
    return(c(title, code, step1, step2, step3, step4, audience_age_10, audience_age_20, audience_age_30,
     audience_age_40, audience_age_50, netizen_score, netizen_count, ntz_male, ntz_female,
      ntz_10, ntz_20, ntz_30, ntz_40, ntz_50, audience_score, audience_count, audience_male,
     audience_female, audience_10, audience_20, audience_30, audience_40, audience_50))
}

info <- vector('list', 100)
top_100_codes <- c(get_movie_code(20220502, 1), get_movie_code(20220502, 2))

for (i in 1:length(top_100_codes)){ # 한 줄씩 차곡차곡 쌓는다.
    info[[i]] <- get_movie_info(top_100_codes[i])
}

final_info <- do.call('rbind', info)
# 컬럼명을 지정함
colnames(final_info) <- c("title","code","genre","country","runtime","release",
        "director","actor","view_class","audience_age_10","audience_age_20",
        "audience_age_30","audience_age_40","audience_age_50",
        "netizen_score","netizen_count","ntz_male","ntz_female","ntz_10",
        "ntz_20","ntz_30","ntz_40","ntz_50","audience_score",
        "audience_count","audience_male","audience_female","audience_10",
        "audience_20","audience_30","audience_40","audience_50")

write.csv(final_info, 'movie.csv', row.names=T) # 최종 csv파일 생성함

필요한 패키지를 로딩중입니다: rvest

Warning message:
"패키지 'rvest'는 R 버전 4.1.3에서 작성되었습니다"
필요한 패키지를 로딩중입니다: tidyverse

Warning message:
"패키지 'tidyverse'는 R 버전 4.1.3에서 작성되었습니다"
-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.6     v dplyr   1.0.8
v tidyr   1.2.0     v stringr 1.4.0
v readr   2.1.2     v forcats 0.5.1

Warning message:
"패키지 'ggplot2'는 R 버전 4.1.3에서 작성되었습니다"
Warning message:
"패키지 'tibble'는 R 버전 4.1.3에서 작성되었습니다"
Warning message:
"패키지 'tidyr'는 R 버전 4.1.3에서 작성되었습니다"
Warning message:
"패키지 'readr'는 R 버전 4.1.3에서 작성되었습니다"
Warning message:
"패키지 'purrr'는 R 버전 4.1.3에서 작성되었습니다"
Warning message:
"패키지 'dplyr'는 R 버전 4.1.3에서 작성되었습니다"
Warning message:
"패키지 'stringr'는 R 버전 4.1.3에서 작성되었습니다"
Warning message:
"패키지 'forcats'는 R 버전 4.1.3에서 작성되었습니다"
-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter()  

In [3]:
final_info

title,code,genre,country,runtime,release,director,actor,view_class,audience_age_10,⋯,ntz_50,audience_score,audience_count,audience_male,audience_female,audience_10,audience_20,audience_30,audience_40,audience_50
클라우스,191613,"애니메이션, 코미디, 가족","스페인, 영국",96분,2019.11.15 개봉,"서지오 파블로스, 카를로스 마르티네즈 로페즈","제이슨 슈왈츠먼, J.K. 시몬스, 라시다 존스더보기",[국내] 전체 관람가,NA,⋯,9.17,NA,NA,NA,NA,NA,NA,NA,NA,NA
그린 북,171539,드라마,미국,130분,2019.01.09 개봉,피터 패럴리,"비고 모텐슨(토니 발레롱가), 마허샬라 알리(돈 셜리 박사)더보기",[국내] 12세 관람가 [해외] PG-13도움말,0%,⋯,9.46,9.55,"2,070",9.52,9.57,9.60,9.50,9.58,9.53,9.58
가버나움,174830,드라마,"레바논, 프랑스",126분,2019.01.24 개봉,나딘 라바키,"자인 알 라피아(자인), 요르다노스 시프로우(라힐)더보기",[국내] 15세 관람가 [해외] R도움말,2%,⋯,9.55,9.54,"1,393",9.39,9.61,9.64,9.56,9.51,9.58,9.49
밥정,186114,"다큐멘터리, 드라마",한국,82분,2020.10.07 개봉,박혜령,임지호(본인)더보기,[국내] 전체 관람가,5%,⋯,9.53,9.70,20,9.75,9.69,10.0,9.43,9.80,10.0,9.75
장민호 드라마 최종회,213746,공연실황,한국,106분,2022.01.24 개봉,NA,장민호더보기,[국내] 전체 관람가,0%,⋯,9.89,9.89,9,9.00,10.0,0.00,0.00,10.0,9.75,10.0
디지몬 어드벤처 라스트 에볼루션 : 인연,192613,"애니메이션, 모험",일본,114분,2021.02.17 개봉,타구치 토모히사,"하나에 나츠키(야가미 타이치), 호소야 요시마사(이시다 야마토), 사카모토 치카(아구몬)더보기",[국내] 12세 관람가,NA,⋯,8.80,NA,NA,NA,NA,NA,NA,NA,NA,NA
베일리 어게인,144906,"모험, 코미디, 드라마",미국,100분,2018.11.22 개봉,라세 할스트롬,"조시 게드(베일리/ 엘리/ 티노/ 버디 목소리), 데니스 퀘이드(이든), K.J. 아파(십대 이든)더보기",[국내] 전체 관람가 [해외] PG도움말,3%,⋯,9.35,9.42,463,9.36,9.44,9.71,9.52,9.36,9.14,9.58
원더,151196,드라마,미국,113분,"2021.02.11 재개봉, 2017.12.27 개봉",스티븐 크보스키,"제이콥 트렘블레이(어기 풀먼), 줄리아 로버츠(이자벨 풀먼), 오웬 윌슨(네이트 풀먼)더보기",[국내] 전체 관람가 [해외] PG도움말,2%,⋯,9.33,9.43,319,9.37,9.46,10.0,9.45,9.47,9.32,9.56
아일라,169240,"드라마, 전쟁","한국, 터키",123분,2018.06.21 개봉,잔 울카이,"김설(아일라), 이스마일 하지오글루(슐레이만)더보기",[국내] 15세 관람가,0%,⋯,9.44,9.13,70,8.81,9.32,0.00,9.45,8.74,9.33,8.17
극장판 바이올렛 에버가든,196843,"애니메이션, 드라마, 판타지",일본,140분,2020.11.12 개봉,이시다테 타이치,"이시카와 유이(바이올렛 에버가든 목소리), 나미카와 다이스케(길베르트 부겐빌리아 목소리)더보기",[국내] 전체 관람가,17%,⋯,9.46,9.68,436,9.69,9.67,9.95,9.66,9.62,9.42,9.86
